In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"

from langchain_openai import AzureOpenAIEmbeddings
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(prompt="Langsmith APIKEY:")

    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")



In [9]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "cv.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

3
Xiao  Liu,  Cloud  Architect/Engineer  
Filderhauptstr,  51,  Stuttgart,  70599,  Germany,  015561665338,  jackliu2006@gmail.com  
  SUMMARY                      
CERTIFICATES    
Sep  2025     Certif

{'producer': 'Skia/PDF m142 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Xiao Liu_Cloud_Arch_EN', 'source': 'cv.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

8

In [20]:
import openai
client = openai.AzureOpenAI(
    azure_endpoint="https://genai-nexus.int.api.corpinter.net/apikey/",
    azure_deployment="gpt-4o-mini",
    api_version="2024-06-01",
    api_key=os.environ["OPENAI_API_KEY"],
    )

embeddings = client.embeddings.create(
        model="text-embedding-3-large",
        input=["Say this is a test", "This is another test"],
    )

NotFoundError: Error code: 404 - {'message': 'no Route matched with those values', 'request_id': 'e82a30736fe89d533098603de5ce174c'}

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://genai-nexus.int.api.corpinter.net/apikey/",
    azure_deployment="gpt-4o-mini",
    api_version="2024-06-01",
    api_key=os.environ["OPENAI_API_KEY"],
    model="text-embedding-3-large"
)
# Generate embedding vectors for all splits
split_texts = [doc.page_content for doc in all_splits]
embedding_vectors = embeddings.embed_documents(split_texts)

NotFoundError: Error code: 404 - {'message': 'no Route matched with those values', 'request_id': '9d459625bb982b5161397c1ea3e4771b'}

In [ ]:
# Display the embedding vectors generated for all splits
for i, vec in enumerate(embedding_vectors):
    print(f"Vector {i}: {vec[:10]}... (length: {len(vec)})")